# Simple HMM in NumPy or PyTorch

In [ ]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl
%run ../acmodel/matrix.py

# Get data

In [ ]:
import pandas as pd
df = pd.read_csv("maxi.tsv", sep="\t", keep_default_na=False)
df

In [ ]:
hmms = []
for wav, sentence in list(zip(df.wav.values, df.sentence.values))[:]:
    print(sentence)
    hmm = HMM(sentence, wav=wav)
    hmm.change_sil_loops()
    hmms.append(hmm)
     #print(hmm)

In [ ]:
hmms[0].mfcc.size()

In [ ]:
for i, hmm in enumerate(hmms):
    print(f"{i}: {hmm.orto}")
    print(hmm.pretty_pron)

In [ ]:
hmm = hmms[23] # 3

In [ ]:
all_mfcc = torch.cat([hmm.mfcc for hmm in hmms])
all_mfcc.size()

In [ ]:
global_mean = all_mfcc.mean(dim=0)
global_mean

In [ ]:
global_var = all_mfcc.std(dim=0)**2  # Equals ((all_mfcc-global_mean)**2).mean(dim=0)
global_var

In [ ]:
import math

In [ ]:
phones = sorted([*{*"".join([hmm.b for hmm in hmms])}]) # set of phones ever seen in hmm.b
" ".join(sorted(phones)) # Never seen: ó

In [ ]:
pi = torch.tensor(math.pi)
pi, pi.dtype

In [ ]:
global_var.prod()*(2*pi)**global_var.size()[0]

In [ ]:
global_var.size()[0]

In [ ]:
global_var.dtype

In [ ]:
(2*math.pi)**global_var.size()[0]

In [ ]:
global_var.prod() * 100000000     # floats, not doubles !!!

# Baum-Welch Reestimation
cleaned version with functions

In [ ]:
print(hmm)

In [ ]:
plot_matrix(hmm.mfcc)

In [ ]:
#ref_means = torch.cat([b_means[ph][None] for ph in phones])

In [ ]:
ref_means = torch.cat([global_mean[None] for ph in phones]) # start every phone model as global mean
# above as matrix, below as dict (we will yet choose which one we use)
b_means = {ph:global_mean for ph in phones} # start every phone model as global mean



In [ ]:
import math  # for pi

def compute_b(ref_means, ref_vars, means):
    """
    Compute probability b() for every combination of reference mean
    (one reference per phome) and pronunciation frame (0.01s segment
    described by cepstrum).
    The ref_vars argument can be either per-phone (as ref_means) or
    common for all phones (in this case it is broadcasted).
    """
    sum = ( (ref_means[None]-means[:,None])**2/ref_vars ).sum(dim=2)

    pi = torch.tensor(math.pi)
    n = global_var.size()[0]
    #g = (global_var.prod()*(2*pi)**n)**-0.5
    g = 1
    b_probs = g * (-0.5*sum).exp()
    return b_probs

In [ ]:
def compute_hmm_b(hmm, means_dict):
    """
    For a sentence hmm model with an attached mfcc, compute b() values
    for every sound frame and every model state.
    """
    b_set = sorted({*hmm.b}) # which phone's b()s will we need
    #print(b_set)
    
    b_set_means = torch.cat([b_means[ph][None] for ph in b_set]) # matrix of all needed means

    #b_means_cat = torch.cat([b_means[ph][None] for ph in hmm.b]) # means repeated as hmm.b dictates
    cb = compute_b(b_set_means, global_var, hmm.mfcc) # compute all needed b()s but each only once
    #print(cb.size())
    # Now repeat each b() column as needed for this hmm
    
    ph_to_i = {ph:i for i, ph in enumerate(b_set)} # map phone to column
    #print(f"{ph_to_i=}")
    
    idx = torch.tensor([ph_to_i[ph] for ph in hmm.b])
    #print(f"{idx=}")
    
    return(cb[:, idx]) # repeat each b() column as needed

print(hmm.b)
b = m(compute_hmm_b(hmm, b_means))
    

In [ ]:
b.size()

In [ ]:
colors = ["yellow","red",(0.5,0.95,1),(0.3,0.4,1),"gray","black","black", "black"]
cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
#plot_matrix(cr, figsize=(19.86, 19.86), cmap=cmap1)
plot_matrix(compute_hmm_b(hmm, b_means), cmap=cmap1)

In [ ]:
#plot_wavfile(hmm.wav)

In [ ]:
base = 1000

In [ ]:

def compute_alpha(hmm, b):
    """
    Compute alpha for hmm with mfcc, using b matrix made with corrent phone models
    """
    A = m(hmm.A)
    tmax = hmm.mfcc.size()[0]
    len_x = A.size()[0]
    x_list = [1]+[0]*(len_x-1)
    x_m = m([x_list])
    exponent = 0
    # allocate space for mantissa-like (kept in range) and row-exponent values
    alpha_m = m.rowlist((tmax,len_x))
    alpha_exp = m.rowlist((tmax,1))

    for row in range(tmax):
        while x_m.max()<1/base: # renormalize and remember power of base used
            x_m *= base
            exponent -= 1
        alpha_exp[row] = exponent
        alpha_m[row] = x_m
        x_m = x_m@A*b[row]
    return alpha_m, alpha_exp

In [ ]:
def compute_beta(hmm, b):
    """
    Compute beta for hmm with mfcc, using b matrix made with corrent phone models
    """
    At = m(hmm.A).T()
    tmax = hmm.mfcc.size()[0]
    len_x = At.size()[0]
    x_list = [1]+[0]*(len_x-1)
    x_m = m([list(reversed(x_list))])
    exponent = 0
    beta_m = m.rowlist((tmax,len_x))
    beta_exp = m.rowlist((tmax,1))

    for row in range(tmax-1,0-1,-1):
        beta_m[row] = x_m
        beta_exp[row] = exponent
        x_m = x_m@At*b[row]
        while x_m.max()<1/base: # renormalize and remember power of base used
            x_m *= base
            exponent -= 1
    return beta_m, beta_exp




In [ ]:
#NOTE: We could in fact get rid of all the exponents in alpha, beta and L cause 
# the final normalization will throw them anyway!
def compute_normalized_L(hmm, b):
    alpha_m, alpha_exp = compute_alpha(hmm, b)
    beta_m, beta_exp = compute_beta(hmm, b)
    L_m = alpha_m*beta_m # this is .*
    L_exp = alpha_exp+beta_exp
    L_exp += float(-L_exp.max())
    # re-normalize L - in fact not needed, we will normalize to sum=1 in rows anyway
    tmax = L_m.size()[0]
    for row in range(tmax):
        while float(L_exp[row].val)<0: # renormalize and remember power of base used
            L_m[row] *= 1/base  # invokes setitem which converts L_m to dense
            L_exp[row] += 1
    X = L_m.val * (1/L_m.val.sum(dim=1))[:,None]
    return X # Normalized L with sum=1 for each row. Only good for alignment (recognition score lost)


In [ ]:
def add_to_distribution_statistics(acc_means, acc_weights, hmm, L):
    """
    Fore one training sentence aligned with its hmm using the current phone models,
    accumulate statistics for later computition of the new phone models.
    The 'L' matrix is expected to be normalized to sum 1 in each row.
    Columns of the L matrix correspond to elements of hmm.b (phone states).
    Rows of the L matrix correspond to rows in hmm.mfcc (0.01s time frames).
    """
    means = (L[:,None]*hmm.mfcc[:,:,None]).sum(dim=0) # sizes like [363, 1, 35] and [363, 13, 1]
    #print(means.size())
    # yet to be summed to phonemes
    weights = L.sum(dim=0)
    #print(weights.size())
    for i, phone in enumerate(hmm.b): # now b is string, may be a list of strings later
        if phone not in acc_means:
            acc_means[phone] = 0 # will be broadcast to the right size
            acc_weights[phone] = 0
        acc_means[phone] += means[:,i]
        acc_weights[phone] += weights[i]


In [ ]:
# Init B-W:

b_means = {ph:global_mean for ph in phones} # start every phone model as global mean

acc_means = {} # can as well be a vector of numbers (later)
acc_weights = {}


In [ ]:
%%time
acc_means = {} # can as well be a vector of numbers (later)
acc_weights = {}
failed = 0
for i, hmm in enumerate(hmms):
    #b = m(compute_hmm_b(hmm, b_means))
    b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))
    L = compute_normalized_L(hmm, b)
    #print(L.sum(dim=0))
    if L.sum().isnan():
        failed += 1
        print(f"Failed hmm alignment, {i=}, {failed/(i+1)*100} % failed so far")
    else:
        add_to_distribution_statistics(acc_means, acc_weights, hmm, L)
    
#b_means = {} keep prev value if we have nothing better (phone was only in failed alignments)
for phone in acc_means:
    if acc_weights[phone]==0:
        print(f"Zero weight for phone {phone}, skipping update")
        continue
    b_means[phone] = acc_means[phone]/acc_weights[phone]

1st iteration: mostly failed alignment (~90% ?)
2nd iteration: ~13% initially failing, better at iter. end: Failed hmm alignment, i=10748, 7.07042515582845 % failed so far
3rd: i=10634, 1.8617771509167842 % failed so far
4th: i=10523, 2.309007981755986 % failed so far   LOOKS MOSTLY CONVERGED
5th: i=10523, 2.451539338654504 % failed so far   CPU times: user 1h 21min 39s, sys: 31 s, total: 1h 22min 10s Wall time: 10min 22s

MFCC_D_A first iter:
CPU times: user 1h 57min 30s, sys: 45.7 s, total: 1h 58min 15s
Wall time: 14min 47s

In [ ]:
# after 1 iters
b_means

In [ ]:
for iter in range(4):
    print(f"{iter=}")
    acc_means = {} # can as well be a vector of numbers (later)
    acc_weights = {}

    for i, hmm in enumerate(hmms):
        #b = m(compute_hmm_b(hmm, b_means))
        b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))
        L = compute_normalized_L(hmm, b)
        #print(L.sum(dim=0))
        if L.sum().isnan():
            print(f"Failed hmm alignment, {i=}")
        else:
            add_to_distribution_statistics(acc_means, acc_weights, hmm, L)
        
    #b_means = {} keep prev value if we have nothing better (phone was only in failed alignments)
    for phone in acc_means:
        if acc_weights[phone]==0:
            print(f"Zero weight for phone {phone}, skipping update")
            continue
        b_means[phone] = acc_means[phone]/acc_weights[phone]

In [ ]:
# After 5 more iters:
b_means

In [ ]:
hmms = []
for wav, sentence in list(zip(df.wav.values, df.sentence.values))[:]:
    print(sentence)
    hmm = HMM(sentence, wav=wav)
    #hmm.change_sil_loops()
    hmms.append(hmm)

In [ ]:
for iter in range(5):
    print(f"{iter=}")
    acc_means = {} # can as well be a vector of numbers (later)
    acc_weights = {}

    for i, hmm in enumerate(hmms):
        #b = m(compute_hmm_b(hmm, b_means))
        b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))
        L = compute_normalized_L(hmm, b)
        #print(L.sum(dim=0))
        if L.sum().isnan():
            print(f"Failed hmm alignment, {i=}")
        else:
            add_to_distribution_statistics(acc_means, acc_weights, hmm, L)
        
    #b_means = {} keep prev value if we have nothing better (phone was only in failed alignments)
    for phone in acc_means:
        if acc_weights[phone]==0:
            print(f"Zero weight for phone {phone}, skipping update")
            continue
        b_means[phone] = acc_means[phone]/acc_weights[phone]

In [ ]:
# After 15 iters:
b_means

In [ ]:
for iter in range(5):
    print(f"{iter=}")
    acc_means = {} # can as well be a vector of numbers (later)
    acc_weights = {}

    for i, hmm in enumerate(hmms):
        #b = m(compute_hmm_b(hmm, b_means))
        b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))
        L = compute_normalized_L(hmm, b)
        #print(L.sum(dim=0))
        if L.sum().isnan():
            print(f"Failed hmm alignment, {i=}")
        else:
            add_to_distribution_statistics(acc_means, acc_weights, hmm, L)
        
    #b_means = {} keep prev value if we have nothing better (phone was only in failed alignments)
    for phone in acc_means:
        if acc_weights[phone]==0:
            print(f"Zero weight for phone {phone}, skipping update")
            continue
        b_means[phone] = acc_means[phone]/acc_weights[phone]

In [ ]:
# After 20 iters:
b_means

In [ ]:
hmm = hmms[5]
print(hmm)
b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))
L = compute_normalized_L(hmm, b)

In [ ]:
i = L.max(1).indices

"".join([hmm.b[ii] for ii in i])

In [ ]:
i = L.max(1).indices

"".join([hmm.b[ii] for ii in i])

In [ ]:
i = L.max(1).indices

"".join([hmm.b[ii] for ii in i])

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_fun(L)

In [ ]:
plot_wavfile(hmm.wav)

In [ ]:
b_means

In [ ]:
b_means

In [ ]:
acc_weights['G'].isnan()

In [ ]:
acc_weights

In [ ]:
print(hmms[-5])

In [ ]:
plot_wavfile(hmms[-5].wav)

In [ ]:
plot_matrix(hmms[-5].mfcc)

In [ ]:
L.sum(dim=0)

In [ ]:
plot_fun(b.val[:,0])

In [ ]:
plot_fun(L)

In [ ]:
plot_wavfile(hmm.wav)

In [ ]:
print(hmm)

In [ ]:
X = (L_m*(1/float(L_m.max()))).val
# now sum of any row in X is 1, we will use these probabilities to compute means

In [ ]:
L_m.val.sum(dim=1)[:,None].size()

In [ ]:
L_m.val.size()

In [ ]:
#X = L_m.val / L_m.val.sum(dim=1)[:,None]


X = L_m.val * (1/L_m.val.sum(dim=1))[:,None]


In [ ]:
plot_matrix(L)

In [ ]:
hmm.mfcc.size()

In [ ]:
L_m.val.max(dim=1)

In [ ]:
plot_fun(X)

In [ ]:
plot_wavfile(hmm.wav)

In [ ]:
plot_fun(b.val)

In [ ]:
b

In [ ]:
b.size()

In [ ]:
print(hmm)

# ============ OLD AREA ============

In [ ]:
import torchaudio
mfcc = hmm.mfcc

In [ ]:
if False: # fake values for testing only
    A = m([[0.7,0.3,0,0],
           [0,0.9,0.1,0],
           [0,0,0.8,0.2],
           [0,0,0,1]])
    tmax = int(363/35 * len_x)

In [ ]:
A = m(hmm.A)
A.size()[0]

In [ ]:
mfcc.size()[0]

In [ ]:
len_x = A.size()[0]
tmax = mfcc.size()[0] # recording length in frames

In [ ]:
# FAKE b VALUES for the first pass AND also for testing
b = m.zeros((tmax,len_x)) + 0.1

# B-W iteration repeats here

In [ ]:
# where we can be
#x_m = m([[1,0,0,0]])
x_list = [1,0,0,0]
x_list = [1]+[0]*(A.size()[0]-1)
x_m = m([x_list])
x_rev = m([list(reversed(x_list))])

In [ ]:
x_m.size()

In [ ]:
len_x = x_m.size()[1]

# allocate space for mantissa-like (kept in range) and row-exponent values
alpha_m = m.rowlist((tmax,len_x))
alpha_exp = m.rowlist((tmax,1))

exponent = 0
base = 1000 # base for separation to exp and m (somewhere around 5e-324 starts underflow)

In [ ]:
#x_m = m([[1,0,0,0]]) 

In [ ]:
%%time
#x_m = m([[1,0,0,0]]) # mantissa-like moderated value (kept in range)
for row in range(tmax):
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1
    alpha_exp[row] = exponent
    alpha_m[row] = x_m
    x_m = x_m@A*b[row]   # FINETUNE WHICH row's b IS USED (ALSO FOR beta)

### Size 14000 x 4
* GPU: 7.01 torch sparse coo, NotImpl torch sparse csr, 0.384 torch dense
* CPU: 1.54 torch sparse coo, 0.210 torch sparse csr, 0.214 torch dense

### Size 363 x 35
* GPU: ??? torch sparse coo, NotImpl torch sparse csr, 0.3770 torch dense
* CPU: ??? torch sparse coo, NotImpl torch sparse csr, 0.0054 torch dense

715 15.4 CPU 391 GPU

1420 33  CPU 434 GPU

2831  78.5 CPU 486 GPU

5652  209  CPU  612 GPU

11294  529 CPU  885 GPU

22578    5000 CPU   1860 GPU

45146    120000 CPU  6110 GPU   unusable GPU coo   not impl CPU csr   unusable CPU coo


In [ ]:
# Now we will go backward
At = A.T()

In [ ]:
beta_m = m.rowlist((tmax,len_x))
beta_exp = m.rowlist((tmax,1))

exponent = 0

In [ ]:
#x_m = m([[0,0,0,1]])
x_m = x_rev
for row in range(tmax-1,0-1,-1):
    beta_m[row] = x_m
    beta_exp[row] = exponent
    x_m = x_m@At*b[row]
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1

In [ ]:
L_m = alpha_m*beta_m # this is .*

L_exp = alpha_exp+beta_exp

L_exp += float(-L_exp.max())

# re-normalize L
for row in range(tmax):
    while float(L_exp[row].val)<0: # renormalize and remember power of base used
        L_m[row] *= 1/base  # invokes setitem which converts L_m to dense
        L_exp[row] += 1

In [ ]:
L_m.size()

In [ ]:
# test - should be the same now
L_exp.min(), L_exp.max()

In [ ]:
L_m.max() # happens to be also equal to sum of any row in L_m

In [ ]:
(b.val==0).sum()

In [ ]:
b

In [ ]:
b.val.clamp(min=0.01).min()

In [ ]:
X = (L_m*(1/float(L_m.max()))).val
# now sum of any row in X is 1, we will use these probabilities to compute means

In [ ]:
X.size()

In [ ]:
scale = 1/X.max(1).values
Y = (X.T*scale).T

In [ ]:
X.sum(dim=1).size()

In [ ]:
means = (X[:,None]*mfcc[:,:,None]).sum(dim=0) # sizes like [363, 1, 35] and [363, 13, 1]
means.size()
# yet to be summed to phonemes

In [ ]:
weights = X.sum(dim=0)
weights

In [ ]:
weights = X.sum(dim=0)
weights

In [ ]:
plot_matrix((means/weights).T)

In [ ]:
# put this cell up
acc_means = {} # can as well be a vector of numbers (later)
acc_weights = {}

In [ ]:
means.size()

In [ ]:
for i, phone in enumerate(hmm.b): # now b is string, may be a list of strings later
    if phone not in acc_means:
        acc_means[phone] = 0 # will be broadcast to the right size
        acc_weights[phone] = 0
        
    acc_means[phone] += means[:,i]
    acc_weights[phone] += weights[i]


In [ ]:
b_means = {}
for phone in acc_means:
    b_means[phone] = acc_means[phone]/acc_weights[phone]

In [ ]:
b_means

In [ ]:
b

In [ ]:
b.size(), mfcc.size()

In [ ]:
b_means_cat = torch.cat([b_means[phone][None] for phone in hmm.b]) # means repeated as hmm.b dictates


In [ ]:
b_means_cat[None].size(), mfcc[:,None].size()

In [ ]:
new_b = ((b_means_cat[None]-mfcc[:,None])**2).sum(dim=2)

In [ ]:
plot_matrix(new_b)

In [ ]:
new_b.size()

In [ ]:
new_b

In [ ]:
plot_fun(-new_b)

In [ ]:
#plot_fun(torch.nn.Softmax(-new_b).float())

plot_fun(torch.nn.Softmax(dim=1)(-new_b/1e2))



In [ ]:
b.size()

In [ ]:
new_b.size()

In [ ]:
b = m(torch.nn.Softmax(dim=1)(-new_b/1e2).clamp(min=0.01))

# B-W iteration can go back here. Below just FYI

In [ ]:
b

In [ ]:
b.min()

In [ ]:
plot_fun(mfcc[:,0])

In [ ]:
plot_matrix(mfcc, figsize=(20,1))

In [ ]:
plot_fun(X)

In [ ]:
plot_fun(Y)

In [ ]:
X

In [ ]:
i = X.max(1).indices

"".join([hmm.b[ii] for ii in i])

In [ ]:
hmm.wav

In [ ]:
plot_fun(mfcc[:,0])

In [ ]:
plot_matrix(Y,cmap='viridis', figsize=(20,2), footer=True)

In [ ]:
plot_matrix(alpha_m.val,cmap='viridis', figsize=(20,2), footer=True)

In [ ]:
plot_matrix(beta_m.val,cmap='viridis', figsize=(20,2), footer=True)

'spectral' is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r'

In [ ]:
plot_matrix(mfcc, figsize=(20,1))

In [ ]:
mfcc.size(), mfcc[None].size(), mfcc[:, None].size()

In [ ]:
d = mfcc[None] - mfcc[:, None]

In [ ]:
d.size()

In [ ]:
dd = d ** 2

In [ ]:
dd.size()

In [ ]:
global_var.size()

In [ ]:
dd_v = dd / (global_var**2)

ddd_v = dd_v.sum(2)

In [ ]:
ddd = dd.sum(2)

In [ ]:
#plot_matrix(ddd, cmap='hsv')
#plot_matrix(ddd, cmap=['twilight_shifted','twilight','gist_ncar','nipy_spectral','turbo','jet','rainbow','gist_rainbow','ocean','terrain'][-1]   )
#plot_matrix(ddd, cmap=['twilight_shifted', 'gist_stern', 'gnuplot', 'gnuplot2', 'RdBu'][-1]   )
#plot_matrix(ddd, cmap=['Spectral'][-1]   )
#plot_matrix(ddd, cmap=['viridis', 'plasma', 'inferno','magma','cividis'][-1]   )


from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


colors = ["cyan", "darkorange", "gold", "lawngreen", "lightseagreen", "black"]

colors = ["black", "yellow", "red", "cyan", "darkorange", "gold", "lawngreen", "lightseagreen", "black"]
colors = [(1,1,0), (0,1,0), (1,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0),
          (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0)]

colors = [(1,0.8,0.6), (1,1,1), (0.7,0.7,0.7), (0.5,0.5,0.5), (0.3,0.3,0.3), (0.3,0.3,0.5), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0),
          (0,0,0), (0,0,0), (1,0,0), (0,0,0), (0,0,0), (0,0,0), (0.5,0,0), (0,0,0), (0,0,0)]


colors = ["red","yellow","green","blue","black", "darkblue"]
colors = ["lightgreen","green","red","yellow","green","blue","black", "darkblue"]
colors = ["lightgreen","green","yellow","red","green","blue","black", "darkblue"]
colors = ["white","green","yellow","red","cyan","blue","black", "darkblue"]

colors = list(reversed(["white","green","yellow","red","cyan","blue","black", "darkblue"]))

colors = ["cyan","blue","yellow","red","gray","blue","black", "darkblue"]
colors = ["cyan","blue","yellow","red","gray","black","black", "black"]
colors = [(0.7,0.9,1),"blue","yellow","red","gray","black","black", "black"]
colors = [(0.7,0.9,1),(0.3,0.4,1),"yellow","red","gray","black","black", "black"]
colors = [(0.5,0.95,1),(0.3,0.4,1),"yellow","red","gray","black","black", "black"]

colors = ["yellow","red",(0.5,0.95,1),(0.3,0.4,1),"gray","black","black", "black"]

cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
plot_matrix(ddd, cmap=cmap1, figsize=(19.86, 19.86))


In [ ]:
plot_matrix(ddd_v, cmap=cmap1, figsize=(19.86, 19.86))

In [ ]:
plot_wavfile(hmm.wav)

In [ ]:
waveform, sample_rate = torchaudio.load(hmm.wav)
plot_specgram(waveform, sample_rate)

In [ ]:
print(hmm)

# Compute new means based on L

In [ ]:
# Mean
#tensor.mean()

# Standard Deviation
#tensor.std()

In [ ]:
len(hmm.b), len({*hmm.b})

In [ ]:
b